In [1]:
import requests
import json
import time
import os
import pandas as pd

In [89]:
def getPage(page = 0):
    params = {
        'text': 'NAME:Web - дизайнер', # - текст запроса 
        'area': 1, # город поиска
        'page': page, # индекс страницы поиска
        'per_page': 100 # число вакансий на странице
    }
    
    
    req = requests.get('https://api.hh.ru/vacancies', params) 
    data = req.content.decode() 
    req.close()
    return data

In [90]:
for page in range(0, 20):
    
    jsObj = json.loads(getPage(page))
    
    # Сохраняем файлы в папку .\paginations
    nextFileName = './paginations/{}.json'.format(len(os.listdir('./paginations')))

    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False))
    f.close()
    
    # проверка на последнюю страницу, если вакансий меньше 2000
    if (jsObj['pages'] - page) <= 1:
        break
    
    time.sleep(0.25)
    
print('Старницы поиска собраны')

Старницы поиска собраны


In [91]:

for fl in os.listdir('./paginations'):
    f = open('./paginations/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
    jsonObj = json.loads(jsonText)
    
    for v in jsonObj['items']:
        
        # Обращаемся к API и получаем детальную информацию по конкретной вакансии
        req = requests.get(v['url'])
        data = req.content.decode()
        req.close()
        
        # Создаем файл в формате json с идентификатором вакансии в качестве названия
        # Записываем в него ответ запроса и закрываем файл
        fileName = './vacations/{}.json'.format(v['id'])
        f = open(fileName, mode='w', encoding='utf8')
        f.write(data)
        f.close()
        
        time.sleep(0.25)
        
print('Вакансии собраны')

Вакансии собраны


In [92]:
# Создаем списки для столбцов таблицы vacancies
IDs = [] # Список идентификаторов вакансий
names = [] # Список наименований вакансий
descriptions = [] # Список описаний вакансий

# Создаем списки для столбцов таблицы skills
skills_vac = [] # Список идентификаторов вакансий
skills_name = [] # Список названий навыков

# В выводе будем отображать прогресс
cnt_docs = len(os.listdir('./vacations'))
i = 0

In [93]:
# Проходимся по всем файлам в папке vacancies
for fl in os.listdir('./vacations'):
    
    # Открываем, читаем и закрываем файл
    f = open('./vacations/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
    
    # Текст файла переводим в справочник
    jsonObj = json.loads(jsonText)
    
    if 'id' in jsonObj:
        IDs.append(jsonObj['id'])
        names.append(jsonObj['name'])
        descriptions.append(jsonObj['description'])
    
        # Т.к. навыки хранятся в виде массива, то проходимся по нему циклом
        for skl in jsonObj['key_skills']:
            skills_vac.append(jsonObj['id'])
            skills_name.append(skl['name'])
    
        # Увеличиваем счетчик обработанных файлов на 1, очищаем вывод ячейки и выводим прогресс
        i += 1
        display.clear_output(wait=True)
        display.display('Готово {} из {}'.format(i, cnt_docs))

'Готово 128 из 190'

In [94]:
# Создаем пандосовский датафрейм, который затем сохраняем в БД в таблицу vacancies
df = pd.DataFrame({'id': IDs, 'name': names, 'description': descriptions})


In [95]:
df.head()

,id,name,description
0,87395444,Бизнес аналитик (Web),<p><strong>Banks Soft Systems — </strong>аккре...
1,87743438,Веб-дизайнер,<p>Привет!</p> <p>Мы ищем в свою команду <stro...
2,90151202,UI graphic web designer,<p><em>Мы занимаемся разработкой и поддержкой ...
3,90340413,Дизайнер-верстальщик/Web Дизайнер,<p>Ведем поиск дизайнера-верстальщика в крупну...
4,90385162,Web-дизайнер,<p>Мы ищем <strong>Web-дизайнера</strong> для ...


In [87]:
df.to_csv('.\api_hh\vacations_design.csv', index=False)